In [1]:
import numpy as np
from intcode_computer import  intcode_computer

from collections import deque

from PIL import Image
import imageio

import matplotlib.pyplot as plt
from matplotlib.animation import ArtistAnimation
%matplotlib inline

In [2]:
#Check if position is traversable
def good_pos(graph, pos):
    x = pos[0]
    y = pos[1]
    if x < 0 or x >= len(graph):
        return False
    if y < 0 or y >= len(graph[x]):
        return False
    value = graph[x][y]
    return value != 0

#Breadth-First Search
def BFS(graph, start, end=None, get_path=False, get_dist=False):
    dx = [-1, 1]
    dy = [-1, 1]
    queue = deque([start])
    dist = {start: 0}
    
    while len(queue):
        cur_pos = queue.popleft()
        cur_dist = dist[cur_pos]
        if cur_pos == end:
            break
        for i in range(0, 2):
            nxt_dist = cur_dist + 1
            #move in x
            nxt_pos = (cur_pos[0]+dx[i], cur_pos[1])
            if good_pos(graph, nxt_pos) and nxt_pos not in dist.keys():
                queue.append(nxt_pos)
                dist[nxt_pos] = nxt_dist
            #move in y
            nxt_pos = (cur_pos[0], cur_pos[1]+dy[i])
            if good_pos(graph, nxt_pos) and nxt_pos not in dist.keys():
                queue.append(nxt_pos)
                dist[nxt_pos] = nxt_dist
                
    max_dist = 0
    for key in dist.keys():
        if dist[key] > max_dist:
            max_dist = dist[key]
    
    if get_path and end is not None:
        path = [end]
        test_dist = max_dist
        dxy = [[1,0],[-1,0],[0,1],[0,-1]]
        while path[-1] != start:
            for delta in dxy:
                test = (path[-1][0]+delta[0],path[-1][1]+delta[1])
                if test in dist and dist[test] < test_dist:
                    path.append(test)
                    test_dist -= 1
                    break
        return path
    
    if get_dist:
        return dist, max_dist
    
    return max_dist

In [3]:
#Map hull

#input
#1 - North
#2 - South
#3 - East
#4 - West
#output
#0 - wall
#1 - empty
#2 - target
#3 - start
#4 - current position

cpu = intcode_computer(15)

position = [19,19]
start = [19,19]
go_back = False
hull = [[[19,19], 3]]
last = 1
dirs = [1, 3, 2, 4]
move = 0
oxygen = []

frames = []
colours = [[66,21,10],[232,236,251],[97,149,207],[220,5,12],[78,178,101]]

show_hull = np.ones((41, 41, 3),dtype=np.int)
show_hull[:,:,:] *= 119
for j in range(0, 3):
    show_hull[start[0],start[1],j] = colours[3][j]

#Start frame
image = Image.fromarray(show_hull.astype('uint8'), mode='RGB')
image = image.resize((41*8,41*8), resample=Image.NEAREST)
image.save('./frames/day15_'+str(0)+'.png')
frames.append(np.array(image))

#Wall hug left wall
i = 0
while True: 
    if last == 0:
        move += 1
        if move == 4:
            move = 0
    else:
        move -= 1
        if move == -1:
            move = 3
    
    if move == 0:
        new_position = [position[0]+1,position[1]]
    elif move == 1:
        new_position = [position[0],position[1]+1]
    elif move == 2:
        new_position = [position[0]-1,position[1]]
    else:
        new_position = [position[0],position[1]-1]
    
    last = cpu.run_program(dirs[move], pause_on_out=True, verbose=False)[0]

    if [new_position, last] not in hull:
        hull.append([new_position, last])
    if last == 1 or last == 2:
        position = new_position
        
    for j in range(0, len(hull)):
        x = hull[j][0][0]
        y =  hull[j][0][1]
        c = colours[hull[j][1]]
        for k in range(0, 3):
            show_hull[x,y,k] = c[k]
    for j in range(0, 3):
        show_hull[position[0],position[1],j] = colours[4][j]
    image = Image.fromarray(show_hull.astype('uint8'), mode='RGB')
    image = image.resize((41*8,41*8), resample=Image.NEAREST)
    image.save('./frames/day15_'+str(i+1)+'.png')
    frames.append(np.array(image))
    
    if go_back and position[0] == start[0] and position[1] == start[1]:
        break
    elif go_back == False and (position[0] != start[0] or position[1] != start[1]):
        go_back = True
    
    i += 1
    
#Pause frames
image = Image.fromarray(show_hull.astype('uint8'), mode='RGB')
image = image.resize((41*8,41*8), resample=Image.NEAREST)
ttl_frames = i+1
ned_frames = 60-(ttl_frames%60)
for i in range(0, ned_frames):
    image.save('./frames/day15_'+str(ttl_frames+i+1)+'.png')
    frames.append(np.array(image))

ttl_frames += i

imageio.mimsave('./day15-vis.gif', frames, duration=1/60)

In [4]:
#Reset coordinate system
x_min = 0
x_max = 0
y_min = 0
y_max = 0
for i in range(0, len(hull)):
    if hull[i][0][0] < x_min:
        x_min = hull[i][0][0]
    if hull[i][0][0] > x_max:
        x_max = hull[i][0][0]
    if hull[i][0][1] < y_min:
        y_min = hull[i][0][1]
    if hull[i][0][1] > y_max:
        y_max = hull[i][0][1]

for i in range(0, len(hull)):
    hull[i][0][0] -= x_min
    hull[i][0][1] -= y_min
    if hull[i][1] == 2:
        end = hull[i][0]
        print('Oxygen at', end)

x_max -= x_min
y_max -= y_min
start = [hull[0][0][0]-x_min,hull[0][0][1]-y_min]
print('Start at',  start)

Oxygen at [35, 3]
Start at [19, 19]


In [5]:
#Convert hull for maze traversal
new_hull = np.ones((x_max+1, y_max+1),dtype=np.int)

for i in range(0, len(hull)):
    new_hull[hull[i][0][0], hull[i][0][1]] = hull[i][1]

#From start to end
oxy_path = BFS(new_hull, tuple(start), tuple(end), True)

#Draw path
for i in range(len(oxy_path)-1, -1, -1):
    ttl_frames += 1
    x = oxy_path[i][0]
    y = oxy_path[i][1]
    for j in range(0, 3):
        show_hull[x,y,j] = colours[4][j]
    image = Image.fromarray(show_hull.astype('uint8'), mode='RGB')
    image = image.resize((41*8,41*8), resample=Image.NEAREST)
    image.save('./frames/day15_'+str(ttl_frames)+'.png')
    frames.append(np.array(image))
    
#Undraw path
for i in range(len(oxy_path)-1, 0, -1):
    ttl_frames += 1
    x = oxy_path[i][0]
    y = oxy_path[i][1]
    for j in range(0, 3):
        show_hull[x,y,j] = colours[1][j]
    image = Image.fromarray(show_hull.astype('uint8'), mode='RGB')
    image = image.resize((41*8,41*8), resample=Image.NEAREST)
    image.save('./frames/day15_'+str(ttl_frames)+'.png')
    frames.append(np.array(image))
        
#Pause frames
image = Image.fromarray(show_hull.astype('uint8'), mode='RGB')
image = image.resize((41*8,41*8), resample=Image.NEAREST)
ttl_frames += 1
ned_frames = 60-(ttl_frames%60)
for i in range(0, ned_frames):
    image.save('./frames/day15_'+str(ttl_frames+i+1)+'.png')
    frames.append(np.array(image))

ttl_frames += i

imageio.mimsave('./day15-vis.gif', frames, duration=1/60)

In [6]:
#From end to filled
dist, max_dist = BFS(new_hull, tuple(end), get_dist=True)

for i in range(0, max_dist+1):
    ttl_frames += 1
    for key in dist.keys():
        if dist[key] == i:
            x = key[0]
            y = key[1]
            for j in range(0, 3):
                show_hull[x,y,j] = colours[2][j]
    image = Image.fromarray(show_hull.astype('uint8'), mode='RGB')
    image = image.resize((41*8,41*8), resample=Image.NEAREST)
    image.save('./frames/day15_'+str(ttl_frames)+'.png')
    frames.append(np.array(image))
    
#Pause frames
image = Image.fromarray(show_hull.astype('uint8'), mode='RGB')
image = image.resize((41*8,41*8), resample=Image.NEAREST)
ttl_frames += 1
ned_frames = 60-(ttl_frames%60)
for i in range(0, ned_frames):
    image.save('./frames/day15_'+str(ttl_frames+i+1)+'.png')
    frames.append(np.array(image))

imageio.mimsave('./day15-vis.gif', frames, duration=1/60)